In [1]:
from felibs.feaifuncs import *
from felibs.unit import Unit

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
## Setting Hyperparameters and Chapter Specific Info

starting_location = {'Lyn' : (2,1),
                     'Sain' : (0,4),
                     'Kent' : (0,2)}


item_slot = {'Lyn' : 1,
             'Sain' : 0,
             'Kent' : 2}


train_episodes = 200
max_turns = 12

In [3]:
def take_turn_greedy(units):
    '''
    Takes a turn using the highest-values in the q-table only
    Iterates through every unit on the team
    Assumes every character is alive
    **NEEDS SOME LOGIC REGARDING CHECKING DEATH QUOTES**
    '''
    print('starting turn')
    press_key('w', interrupt=0.5)
    press_key("'")
    press_key('s')
    press_key("'")
    blue, red, turns = grab_board_state()
    if blue < 3:
        return 'Sain'
    else:
        press_key(';')
    time.sleep(0.2)
    for u in units:
        time.sleep(1)
        select_next_unit()

        moving = True
        while moving:
            time.sleep(0.2)

            state_index = u.states_to_indices[u.state]
            best_action = u.indices_to_moves_and_actions[np.argmax(u.qtable[state_index])]

            u.move(best_action)
            time.sleep(0.3)
            opts = grab_options()

            if u.item == 0 and best_action[2] == 'Item':
                best_action = (best_action[0], best_action[1], 'Wait')

            if best_action[2] == 'Attack' and not attack_in_opts(opts):
                best_action = (best_action[0], best_action[1], 'Wait')

            if 'Rescue' in opts or 'RESCUE' in opts or 'ESCUED' in opts:
                best_action = (best_action[0], best_action[1], 'Wait')
                u.act(best_action)
                u.save_qtable(1)
                moving=False
                break
            
            if not opts or not wait_in_opts(opts):
                u.update_qtable_invalid_move()
                #press b and try again
                press_key(';')
                press_key("'")
                time.sleep(0.1)
            else:
                u.act(best_action)
                u.save_qtable(1)
                moving = False
        if best_action[2] == 'Attack':
            for _ in range(2):
                dq = check_death_quote()
                if dq and dq != 'NOT DEATH':
                    for u in units:
                        if dq == u.name:
                            u.update_qtable_death(target=True)
                        else:
                            u.update_qtable_death()
                    return dq

                elif dq == 'NOT DEATH':
                    press_key('enter', 2, 0.3)
                    time.sleep(4)
    
    print('starting break')
    
    enemy_phase_break(8)
    
    dq = check_death_quote()
    print('Output of death quote check:', dq)
    if dq and dq != 'NOT DEATH':
        for u in units:
            if dq == u.name:
                u.update_qtable_death(target=True)
            else:
                u.update_qtable_death()
        return dq

    elif dq == 'NOT DEATH':
        press_key('enter', 2, 0.3)

    return ''

In [4]:
### full episode logic with greedy solve
select_vba()

for episode in range(train_episodes):
    print('EPISODE', episode+1)
    time.sleep(0.3)
    reset_to_ch1()
    units=[]
    #instantiate units, load qtable from file/instantiate if episode==1
    time.sleep(1)
    press_key('w', interrupt=0.5)
    for _ in range(3):
        select_next_unit()
        press_key(';')
        name = grab_name()
        press_key("e")
        stats = grab_stats()

        new_unit = Unit(name, stats, starting_location[name], item_slot[name])
        
#         if episode == 0:
#             new_unit.setup_qtable(15,10)
#             new_unit.initialize_qtable_with_target((13,8))
#         else:
#             new_unit.setup_qtable(15,10)
#             new_unit.load_qtable(1)

        new_unit.setup_qtable(15,10)
        new_unit.load_qtable(1)
        
        units.append(new_unit)
        press_key(';')

    #take turns
    for turn in range(max_turns):
        print('Turn:', turn+1)
        if turn > 1:
            for u in units:
                u.save_qtable(chapter=1)
            
        #take turn
        death = take_turn_greedy(units)
        if death:
            for u in units:
                if u.name == death:
                    u.update_qtable_death(target=True)
                    u.save_qtable(chapter=1)
                else:
                    u.update_qtable_death()
                    u.save_qtable(chapter=1)
            break
    

EPISODE 1
Turn: 1
starting turn
starting break
 
Output of death quote check: NOT DEATH
Turn: 2
starting turn
starting break
 
 
 
Output of death quote check: Sain
EPISODE 2
Turn: 1
starting turn
starting break
 
Output of death quote check: 
Turn: 2
starting turn


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d1f73593a142>", line 42, in <module>
    death = take_turn_greedy(units)
  File "<ipython-input-3-fcd18d3d4a37>", line 32, in take_turn_greedy
    opts = grab_options()
  File "/Users/aaronfrederick/Desktop/Data Science:Programming/Side Projects/FEAI/felibs/feaifuncs.py", line 311, in grab_options
    short_text1 = pytesseract.image_to_string(inverted_grayscale(short_1))
  File "/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/pytesseract/pytesseract.py", line 309, in image_to_string
    }[output_type]()
  File "/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/pytesseract/pytesseract.py", line 308, in <lambda>
    Output.STRING: lambda: run_and_get_output(*args),
  File "/Users/aaronfrederick/Desktop/3venv/lib/python

KeyboardInterrupt: 

In [23]:

select_vba()
press_key('w', interrupt=0.5)
press_key("'")
press_key('s')
press_key("'")
blue, red, turns = grab_board_state()
press_key(':')
if blue < 3:
    print('Sain')

time.sleep(5)

select_next_unit()
units[0].move((3,1,'Wait'))

In [18]:
select_vba()
select_next_unit()

In [6]:
grab_options()

[]

In [11]:
units

In [ ]:
decay = 0.95

for i in range(len(all_actions)):
    state=all_states[i]
    action=all_actions[i]
    
    si = states_to_indices[state]
    ai = actions_to_indices[action]
    
    qtable[si, ai] *= decay**(i+1)

In [8]:
units[1].all_states

[(0, 4), (5, 6), (9, 8)]

In [12]:
state_index = units[1].states_to_indices[(9,8)]

units[1].indices_to_moves_and_actions[np.argmax(units[1].qtable[state_index])]

(7, 0, 'Attack')

In [19]:
units[1].qtable[state_index, np.argmax(units[1].qtable[state_index])]

0.46982919631397935

In [20]:
units[1].qtable[state_index]

array([-0.49954948, -0.49954948, -0.49954948, -0.44446754, -0.44446754,
       -0.44446754, -0.42752858, -0.42752858, -0.42752858, -0.41421356,
       -0.41421356, -0.41421356, -0.39497166, -0.39497166, -0.39497166,
       -0.37349426, -0.37349426, -0.37349426, -0.35566878, -0.35566878,
       -0.35566878, -0.34164079, -0.34164079, -0.34164079, -0.33153031,
       -0.33153031, -0.33153031, -0.35167564, -0.35167564, -0.35167564,
       -0.32542701, -0.32542701, -0.32542701, -0.30280363, -0.30280363,
       -0.30280363, -0.28399714, -0.28399714, -0.28399714, -0.26917722,
       -0.26917722, -0.26917722, -0.25848472, -0.25848472, -0.25848472,
       -0.25202539, -0.25202539, -0.25202539, -0.3151919 , -0.3151919 ,
       -0.3151919 , -0.28399714, -0.28399714, -0.28399714, -0.2563353 ,
       -0.2563353 , -0.2563353 , -0.23244429, -0.23244429, -0.23244429,
       -0.21254702, -0.21254702, -0.21254702, -0.19684269, -0.19684269,
       -0.19684269, -0.18549796, -0.18549796, -0.18549796, -0.17

In [31]:
units[0].all_moves

[(3, 1, 'Wait'), (2, 2, 'Wait')]

In [2]:
#beginning of the episode
select_vba()
reset_to_ch1()


#initialize unit objects

starting_location = {'Lyn' : (2,1),
                     'Sain' : (0,4),
                     'Kent' : (0,2)}


item_slot = {'Lyn' : 1,
             'Sain' : 0,
             'Kent' : 2}

units = []

time.sleep(1)
press_key('w', interrupt=0.5)

for _ in range(3):
    select_next_unit()
    press_key(';')
    name = grab_name()
    press_key("e")
    stats = grab_stats()
    
    new_unit = Unit(name, stats, starting_location[name], item_slot[name])
    new_unit.setup_qtable(15,10)
    new_unit.initialize_qtable_with_target((13,8))
    units.append(new_unit)
    press_key(';')

select_next_unit()
press_key(';')



#play a turn

#moving one person represented by 'u'
"""
BUGS TO FIX IN THIS BLOCK:
- I imagine trade/rescue/drop might have issues
"""

#death quote dictionary
dqd = pickle.load(open('data/death_quotes.pkl', 'rb'))
exploration_rate = 0
n_turns = 5

# dq = check_death_quote(dqd)
# if dq and dq != 'NOT DEATH':
#     for u in units:
#         if dq == u.name:
#             u.update_qtable_death(target=True)
#         else:
#             u.update_qtable_death()

# elif dq == 'NOT DEATH':
#     press_key('enter')
for _ in range(n_turns):
    press_key('w')
    for u in units:
        time.sleep(1)
        select_next_unit()

        moving = True
        while moving:
            time.sleep(0.2)
            exp_exp_tradeoff = random.uniform(0,1)
            if exp_exp_tradeoff > exploration_rate:
                state_index = u.states_to_indices[u.state]
                best_action = u.indices_to_moves_and_actions[np.argmax(u.qtable[state_index])]

                u.move(best_action)
                time.sleep(0.3)
                opts = grab_options()

                if u.item == 0 and best_action[2] == 'Item':
                    best_action = (best_action[0], best_action[1], 'Wait')

                can_attack = attack_in_opts(opts)
                if best_action[2] == 'Attack' and not can_attack:
                    best_action = (best_action[0], best_action[1], 'Wait')

                if not opts or 'Wait' not in opts:
                    u.update_qtable_invalid_move()
                    #press b and try again
                    press_key(';')
                    press_key("'")
                    time.sleep(0.2)
                else:
                    u.act(best_action)
                    moving = False

            else:
                u.move(random_move=True)
                opts = grab_options()
                if not opts:
                    u.update_qtable_invalid_move()
                    #press b and try again
                    press_key(';')
                else:
                    u.act(random_act=True)
                    moving = False
    enemy_phase_break()

/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Lyn has a max movement of 5 and an item in the 1 slot
Sain has a max movement of 7 and an item in the 0 slot
Kent has a max movement of 7 and an item in the 2 slot
Lyn has a max movement of 5 and an item in the 1 slot
Sain has a max movement of 7 and an item in the 0 slot
Kent has a max movement of 7 and an item in the 2 slot
Lyn has a max movement of 5 and an item in the 1 slot


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0ef6224090d0>", line 80, in <module>
    opts = grab_options()
  File "/Users/aaronfrederick/Desktop/Data Science:Programming/Side Projects/FEAI/felibs/feaifuncs.py", line 249, in grab_options
    screen = np.array(ImageGrab.grab(bbox=(0,0,600,400)))
  File "/Users/aaronfrederick/Desktop/3venv/lib/python3.7/site-packages/PIL/ImageGrab.py", line 36, in grab
    subprocess.call(['screencapture', '-x', filepath])
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/subprocess.py", line 306, in call
    return p.wait(timeout=timeout)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/subprocess.py", line 971, in wait
    return self._wait(timeout=timeout)
  File "/Library/Frameworks/Python.framework/Versions/3.

KeyboardInterrupt: 